# Lineup Optimization for Maximizing GD per Stint

This notebook optimizes player lineups for teams (e.g., Canada) to maximize goal differential (GD) per stint, constrained by the total player rating sum ≤ 8 per stint.

In [22]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict

# Define data directory
DATA_DIR = 'data/'

## Section 1: Load and Prepare Data

In [23]:
# Load player data
player_data = pd.read_csv(DATA_DIR + 'player_data.csv')

# Extract normalized stint_gd from player_data
stint_gd = {}
for _, row in player_data.iterrows():
    player = row['player']
    stint_gd[player] = {}
    for i in range(1, 17):
        col = f'stint{i}_gd'
        if col in row.index and not pd.isna(row[col]):
            stint_gd[player][i] = row[col]

# Extract WOWY data
wowy_dict = dict(zip(player_data['player'], player_data['wowy']))

print("Normalized stint_gd and WOWY loaded from player_data")


Normalized stint_gd and WOWY loaded from player_data


## Section 2: Filter for Team (Canada)

In [24]:
# Filter for Canada
team = 'Canada'
team_players = player_data[player_data['team'] == team]

print(f"Team: {team}, Players: {len(team_players)}")

Team: Canada, Players: 12


## Section 3: Optimize Lineups per Stint


In [25]:
# MILP optimization with Gurobi
import gurobipy as gp
from gurobipy import GRB


def optimize_lineup_milp(stint_num, players_df, stint_gd_dict, wowy_dict, rating_limit=8, wowy_factor=1.0):
    # Filter players with stint data
    valid_df = players_df[players_df['player'].apply(lambda p: stint_num in stint_gd_dict.get(p, {}))]
    if valid_df.empty:
        return None, None, None

    players = valid_df['player'].tolist()
    ratings = valid_df.set_index('player')['rating'].to_dict()
    scores = {p: stint_gd_dict[p][stint_num] + wowy_dict.get(p, 0) * wowy_factor for p in players}

    model = gp.Model()
    model.Params.OutputFlag = 0
    x = {p: model.addVar(vtype=GRB.BINARY, name=p) for p in players}

    # Objective
    model.setObjective(gp.quicksum(scores[p] * x[p] for p in players), GRB.MAXIMIZE)
    # Constraints
    model.addConstr(gp.quicksum(x[p] for p in players) == 4)
    model.addConstr(gp.quicksum(ratings[p] * x[p] for p in players) <= rating_limit)

    model.optimize()
    if model.Status != GRB.OPTIMAL:
        return None, None, None

    sel = [p for p in players if x[p].X > 0.5]
    best_score = model.ObjVal
    rating_sum = sum(ratings[p] for p in sel)
    return sel, best_score, rating_sum



In [26]:
# Function to optimize lineup for a stint
def optimize_lineup(stint_num, players_df, stint_gd_dict, wowy_dict, rating_limit=8, wowy_factor=1.0):
    players = players_df['player'].tolist()
    ratings = players_df.set_index('player')['rating'].to_dict()
    
    # Get players with GD for this stint
    valid_players = [p for p in players if stint_num in stint_gd_dict.get(p, {})]
    if len(valid_players) < 4:
        return None, None, None
    
    best_score = float('-inf')
    best_lineup = None
    best_rating_sum = None
    
    # Brute force: try all combinations of 4 players
    for combo in combinations(valid_players, 4):
        rating_sum = sum(ratings[p] for p in combo)
        if rating_sum <= rating_limit:
            # Score = sum of (GD + WOWY * factor)
            score = sum(stint_gd_dict[p][stint_num] + wowy_dict.get(p, 0) * wowy_factor for p in combo)
            if score > best_score:
                best_score = score
                best_lineup = combo
                best_rating_sum = rating_sum
    
    return best_lineup, best_score, best_rating_sum

# Get max stint number
max_stint = max(max(stint_gd[p].keys()) for p in stint_gd if stint_gd[p])

# Optimize for each stint (MILP if available, else brute force)
optimized_lineups = {}
for stint in range(1, max_stint + 1):
    lineup, score, rating = optimize_lineup_milp(stint, team_players, stint_gd, wowy_dict)
    if lineup:
        optimized_lineups[stint] = {'lineup': lineup, 'score': score, 'rating_sum': rating}
    else:
        optimized_lineups[stint] = None

print("Optimization completed with Gurobi MILP")


Optimization completed with Gurobi MILP


## Section 4: Display Results


In [27]:
# Display results
for stint, data in optimized_lineups.items():
    if data:
        print(f"Stint {stint}: Score = {data['score']:.2f}, Rating Sum = {data['rating_sum']:.2f}")
        print(f"  Lineup: {', '.join(data['lineup'])}")
    else:
        print(f"Stint {stint}: No valid lineup found")
    print()


Stint 1: Score = 0.34, Rating Sum = 8.00
  Lineup: Canada_p3, Canada_p6, Canada_p9, Canada_p11

Stint 2: Score = 1.00, Rating Sum = 8.00
  Lineup: Canada_p2, Canada_p5, Canada_p6, Canada_p7

Stint 3: Score = 1.44, Rating Sum = 8.00
  Lineup: Canada_p2, Canada_p3, Canada_p5, Canada_p9

Stint 4: Score = 0.55, Rating Sum = 8.00
  Lineup: Canada_p3, Canada_p6, Canada_p9, Canada_p11

Stint 5: Score = 1.92, Rating Sum = 5.50
  Lineup: Canada_p6, Canada_p9, Canada_p10, Canada_p12

Stint 6: Score = 2.30, Rating Sum = 7.00
  Lineup: Canada_p2, Canada_p8, Canada_p9, Canada_p10

Stint 7: Score = 1.67, Rating Sum = 4.50
  Lineup: Canada_p2, Canada_p6, Canada_p10, Canada_p11

Stint 8: Score = 2.47, Rating Sum = 8.00
  Lineup: Canada_p1, Canada_p6, Canada_p7, Canada_p12

Stint 9: Score = 0.83, Rating Sum = 8.00
  Lineup: Canada_p1, Canada_p9, Canada_p11, Canada_p12

Stint 10: Score = 1.21, Rating Sum = 6.50
  Lineup: Canada_p2, Canada_p6, Canada_p7, Canada_p11

Stint 11: Score = 1.06, Rating Sum = 7